In [1]:
import os
import shutil
from bs4 import BeautifulSoup
import cv2

## DATA EXTRACTION

In [2]:
cwd = os.getcwd()
datasetpath=cwd + '\dataset'

In [6]:
annot = cwd + '\\annot'
annots = os.listdir(annot)
annotations=[]
unique_labels=[]
for a in annots:
    gtBoxes=[]
    labels=[]
    path = annot+"\\"+a
    contents = open(path).read()
    soup = BeautifulSoup(contents, "html.parser")
    w = int(soup.find("width").string)
    h = int(soup.find("height").string)
    for o in soup.find_all("object"):
        label = o.find("name").string.split()[0]
        if label[0] == '"':
            label = label[1:]
        xMin = int(o.find("xmin").string)
        yMin = int(o.find("ymin").string)
        xMax = int(o.find("xmax").string)
        yMax = int(o.find("ymax").string)
        xMin = max(0, xMin)
        yMin = max(0, yMin)
        xMax = min(w, xMax)
        yMax = min(h, yMax)
        labels.append(label)
        gtBoxes.append((xMin, yMin, xMax, yMax))
    unique_labels.extend(labels)
    annotations.append((a.split(".")[0],labels,gtBoxes,w,h))
print(len(annotations))
unique_labels = list(set(unique_labels))
print(len(unique_labels))

47
29


In [8]:
for a in annotations:
    (image_name,object_labels,object_boxes,width,height)=a
    if width > 1024:
        image_path = cwd + "\\images\\"+image_name+".jpg"
        img=cv2.imread(image_path)
        img = cv2.resize(img, (1024,1024),interpolation=cv2.INTER_CUBIC)
        cv2.imwrite(image_path,img)

In [11]:
for a in annotations:
    (image_name,object_labels,object_boxes,width,height)=a
    image_path = cwd + "\\images\\"+image_name+".jpg"
    img=cv2.imread(image_path)
    (new_width,new_height,c) = img.shape 
    scale_x = new_width//width
    scale_y = new_height//height
    for i in range(0,len(object_boxes)):
        (gtStartX, gtStartY, gtEndX, gtEndY) = object_boxes[i]
        gtStartX*=scale_x
        gtStartY*=scale_y
        gtEndX*=scale_x
        gtEndY*=scale_y
        object_boxes[i] = (gtStartX, gtStartY, gtEndX, gtEndY)

## DATA SET GENERATION 

In [29]:
for a in annotations:
    (image_name,object_labels,object_boxes,width,height)=a
    image_path = cwd + "\\images\\"+image_name+".jpg"
    im = cv2.imread(image_path)
    print(image_path)
    nb_objects = len(object_labels)
    for i in range(0,nb_objects):
        label= object_labels[i]
        destination= datasetpath + "\\"+label+"\\"+image_name+str(i)+".jpg"
        (xMin, yMin, xMax, yMax)= object_boxes[i]
        component = im[yMin:yMax,xMin:xMax]
        cv2.imwrite(destination,component)

C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\ACM-109_Bottom.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\ACM-109_Top.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\ArduinoMega_Bottom.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\ArduinoMega_Top.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\Arty_Bottom.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\Arty_Top.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\ATTIOT_Bottom.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\ATTIOT_Top.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\DigitalDiscovery.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\DuetWIFI_Bottom.jpg
C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\images\DuetWIFI_Top.jpg
C:\Users\n

## DATA AUGMENTATION

In [34]:
for l in unique_labels:
    classe = datasetpath+"\\"+l
    files = os.listdir(classe)
    if len(files)<100:
        for i,file in enumerate(files):
            orig= datasetpath + "\\"+l+"\\"+file
            img = cv2.imread(orig)
            img_rotate_90_clockwise = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            cv2.imwrite(orig[:-4]+'rotate_90_clockwise.jpg', img_rotate_90_clockwise)
            img_rotate_90_counterclockwise = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            cv2.imwrite(orig[:-4]+'rotate_90_counterclockwise.jpg', img_rotate_90_counterclockwise)
            img_rotate_180 = cv2.rotate(img, cv2.ROTATE_180)
            cv2.imwrite(orig[:-4]+'rotate_180.jpg', img_rotate_180)
    print(l,len(os.listdir(classe)))

battery 4
electrolytic 246
connector 621
pins 317
component 995
switch 232
led 214
resistor 2191
inductor 276
zener 20
transformer 4
transistor 336
fuse 28
text 9190
ferrite 120
unknown 281
heatsink 16
clock 148
emi 204
pads 332
ic 386
capacitor 2534
display 20
diode 288
test 292
buzzer 4
jumper 340
button 340
potentiometer 28


In [35]:
for l in unique_labels:
    classe = datasetpath+"\\"+l
    files = os.listdir(classe)
    if len(files)<100:
        for file in files:
            orig= datasetpath + "\\"+l+"\\"+file
            img = cv2.imread(orig)
            cv2.imwrite(orig[:-4]+'copy.jpg', img)
    print(l,len(os.listdir(classe)))

battery 8
electrolytic 246
connector 621
pins 317
component 995
switch 232
led 214
resistor 2191
inductor 276
zener 40
transformer 8
transistor 336
fuse 56
text 9190
ferrite 120
unknown 281
heatsink 32
clock 148
emi 204
pads 332
ic 386
capacitor 2534
display 40
diode 288
test 292
buzzer 8
jumper 340
button 340
potentiometer 56


In [38]:
positives = 0
for l in unique_labels:
    classe = datasetpath+"\\"+l
    positives += len(os.listdir(classe))
print(positives)

20131


In [36]:
print(len(annotations))

47


## EXTRACTING NEGATIVE IMAGES

In [13]:
def compute_iou(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])

	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the intersection area
	iou = interArea / float(boxAArea + boxBArea - interArea)

	# return the intersection over union value
	return iou

In [14]:
NEGATIVE_PATH=datasetpath+"\\not_component"

In [15]:
for a in annotations:
    (image_name,object_labels,object_boxes,width,height)=a
    print(image_name)
    image_path = cwd + "\\images\\"+image_name+".jpg"
    nb_objects = len(object_labels)
    image = cv2.imread(image_path)
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    print("extracting rects...")
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    proposedRects= []
    for (x, y, w, h) in rects:
        proposedRects.append((x, y, x + w, y + h))
    negatives = 0
    i=0
    print(len(proposedRects))
    print("choosing rects...")
    while(negatives<400):
        if negatives%40==0:
            print(negatives/4,"% --> ",end="")
        rect = proposedRects[i]
        (propStartX, propStartY, propEndX, propEndY) =rect
        fullOverlap = True
        max_iou = 0.15
        for box in object_boxes:
            max_iou = max(max_iou,compute_iou(box, rect))
            (gtStartX, gtStartY, gtEndX, gtEndY) = box
            fullOverlap = fullOverlap and propStartX >= gtStartX
            fullOverlap = fullOverlap and propStartY >= gtStartY
            fullOverlap = fullOverlap and propEndX <= gtEndX
            fullOverlap = fullOverlap and propEndY <= gtEndY
        roi = None
        outputPath = None
        if not fullOverlap and max_iou <= 0.15:
            roi = image[propStartY:propEndY, propStartX:propEndX]
            filename = image_name+str(negatives)+".png"
            outputPath = os.path.sep.join([NEGATIVE_PATH,filename])
            negatives += 1
        if roi is not None and outputPath is not None:
            cv2.imwrite(outputPath, roi)
        i+=1

ACM-109_Bottom
extracting rects...
6890
choosing rects...
0.0 % --> 10.0 % --> 20.0 % --> 30.0 % --> 40.0 % --> 50.0 % --> 60.0 % --> 70.0 % --> 80.0 % --> 90.0 % --> ACM-109_Top
extracting rects...
7721
choosing rects...
0.0 % --> 10.0 % --> 20.0 % --> 30.0 % --> 40.0 % --> 50.0 % --> 60.0 % --> 70.0 % --> 80.0 % --> 90.0 % --> ArduinoMega_Bottom
extracting rects...
2690
choosing rects...
0.0 % --> 10.0 % --> 20.0 % --> 30.0 % --> 40.0 % --> 50.0 % --> 60.0 % --> 70.0 % --> 80.0 % --> 90.0 % --> ArduinoMega_Top
extracting rects...
5351
choosing rects...
0.0 % --> 10.0 % --> 20.0 % --> 30.0 % --> 40.0 % --> 50.0 % --> 60.0 % --> 70.0 % --> 80.0 % --> 90.0 % --> Arty_Bottom
extracting rects...
9203
choosing rects...
0.0 % --> 10.0 % --> 20.0 % --> 30.0 % --> 40.0 % --> 50.0 % --> 60.0 % --> 70.0 % --> 80.0 % --> 90.0 % --> Arty_Top
extracting rects...
11080
choosing rects...
0.0 % --> 10.0 % --> 20.0 % --> 30.0 % --> 40.0 % --> 50.0 % --> 60.0 % --> 70.0 % --> 80.0 % --> 90.0 % --> ATTI

In [16]:
print(datasetpath)

C:\Users\newgh\1 jupiter\Computer Vision\pcb\pcb detect&classify\dataset


In [18]:
for classe in os.listdir(datasetpath):
    print(classe,len(os.listdir(os.path.join(datasetpath,classe))))

battery 8
button 340
buzzer 8
capacitor 2534
clock 148
component 995
connector 621
diode 288
display 40
electrolytic 246
emi 204
ferrite 120
fuse 56
heatsink 32
ic 386
inductor 276
jumper 340
led 214
not_component 18801
pads 332
pins 317
potentiometer 56
resistor 2191
switch 232
test 292
text 9190
transformer 8
transistor 336
unknown 281
zener 40
